# Fundamentals of Data Analysis, Winter 21/22
- Author: Brendan Tunney
- ID - G00270683

## Importing the CAO data


In [7]:
#Importing libraries

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import re # {1}
import requests as rq #{1} # To retrieve HTTP Data
from bs4 import BeautifulSoup # Used to pull data from HTML
import csv
from unicodedata import normalize